# 一般的 RNN

对于最简单的 RNN，我们可以使用下面两种方式去调用，分别是 `torch.nn.RNNCell()` 和 `torch.nn.RNN()`，这两种方式的区别在于 `RNNCell()` 只能接受序列中单步的输入，且必须传入隐藏状态，而 `RNN()` 可以接受一个序列的输入，默认会传入全 0 的隐藏状态，也可以自己申明隐藏状态传入。

`RNN()` 里面的参数有

input_size 表示输入 $x_t$ 的特征维度

hidden_size 表示输出的特征维度

num_layers 表示网络的层数

nonlinearity 表示选用的非线性激活函数，默认是 'tanh'

bias 表示是否使用偏置，默认使用

batch_first 表示输入数据的形式，默认是 False，就是这样形式，(seq, batch, feature)，也就是将序列长度放在第一位，batch 放在第二位

dropout 表示是否在输出层应用 dropout

bidirectional 表示是否使用双向的 rnn，默认是 False

对于 `RNNCell()`，里面的参数就少很多，只有 input_size，hidden_size，bias 以及 nonlinearity

In [1]:
import torch
from torch.autograd import Variable
from torch import nn

In [2]:
# 定义一个单步的 rnn
rnn_single = nn.RNNCell(input_size=100, hidden_size=200)

In [3]:
params = rnn_single.parameters()

k=0
for i in params:
    l =1
    print "该层的结构："+str(list(i.size()))
    for j in i.size():
        l *= j
    print "参数和："+str(l)
    k = k+l

print "总参数和："+ str(k)

该层的结构：[200L, 100L]
参数和：20000
该层的结构：[200L, 200L]
参数和：40000
该层的结构：[200L]
参数和：200
该层的结构：[200L]
参数和：200
总参数和：60400


In [4]:
# 访问其中的参数
rnn_single.weight_hh

Parameter containing:
1.00000e-02 *
-5.7257  2.5114  0.4516  ...   0.4061  1.8003 -5.3001
-3.3166 -4.2679 -1.8768  ...  -0.1945  6.8448 -6.6519
 5.4659 -4.8286 -5.8837  ...   4.8544  6.4955 -5.8155
          ...             ⋱             ...          
 0.0088 -1.2067  1.0904  ...   0.0771 -2.0546 -1.5899
-1.3983 -4.2236  3.5697  ...   0.7173  1.0943  5.2856
 5.5566 -4.8115 -6.1825  ...   0.6625  0.7682  0.5913
[torch.FloatTensor of size 200x200]

In [5]:
# 构造一个序列，长为 6，batch 是 5， 特征是 100
x = Variable(torch.randn(6, 5, 100)) # 这是 rnn 的输入格式

In [6]:
# 定义初始的记忆状态
h_t = Variable(torch.zeros(5, 200))

In [7]:
# 传入 rnn
out = []
for i in range(6): # 通过循环 6 次作用在整个序列上
    h_t = rnn_single(x[i], h_t)
    out.append(h_t)

In [8]:
h_t

Variable containing:
-0.0284  0.7627  0.0614  ...   0.0200 -0.8227 -0.5590
 0.2402 -0.4158 -0.7800  ...  -0.3841 -0.0210  0.5196
-0.2617 -0.0287 -0.1040  ...  -0.0734  0.4400  0.4516
-0.2044  0.0402  0.3951  ...   0.7965 -0.0212  0.0114
 0.2732  0.2008 -0.2781  ...  -0.4344 -0.2968  0.2427
[torch.FloatTensor of size 5x200]

In [9]:
len(out)

6

In [10]:
out[0].shape # 每个输出的维度

torch.Size([5, 200])

可以看到经过了 rnn 之后，隐藏状态的值已经被改变了，因为网络记忆了序列中的信息，同时输出 6 个结果

下面我们看看直接使用 `RNN` 的情况

In [11]:
rnn_seq = nn.RNN(100, 200, 3)

In [12]:
# 访问其中的参数
#rnn_seq.weight_hh_l0

In [13]:
out, h_t = rnn_seq(x) # 使用默认的全 0 隐藏状态

In [14]:
#h_t

In [15]:
len(out)

6

In [16]:
out.shape

torch.Size([6, 5, 200])

这里的 h_t 是网络最后的隐藏状态，网络也输出了 6 个结果

In [17]:
# 自己定义初始的隐藏状态
h_0 = Variable(torch.randn(3, 5, 200))

这里的隐藏状态的大小有三个维度，分别是 (num_layers * num_direction, batch, hidden_size)

In [18]:
out, h_t = rnn_seq(x, h_0)

In [19]:
#h_t

In [20]:
h_t.shape

torch.Size([3, 5, 200])

In [21]:
out.shape

torch.Size([6, 5, 200])

同时输出的结果也是 (seq, batch, feature)

一般情况下我们都是用 `nn.RNN()` 而不是 `nn.RNNCell()`，因为 `nn.RNN()` 能够避免我们手动写循环，非常方便，同时如果不特别说明，我们也会选择使用默认的全 0 初始化隐藏状态

## LSTM

LSTM 和基本的 RNN 是一样的，他的参数也是相同的，同时他也有 `nn.LSTMCell()` 和 `nn.LSTM()` 两种形式，跟前面讲的都是相同的，我们就不再赘述了，下面直接举个小例子

In [22]:
lstm_seq = nn.LSTM(100, 200, num_layers=2) # 输入维度 100，输出 200，两层

In [23]:
lstm_seq.weight_hh_l0 # 第一层的 h_t 权重

Parameter containing:
 6.8965e-02  4.6445e-02  3.7888e-02  ...   4.0366e-02  1.9477e-02  4.7692e-03
-3.9616e-02 -5.2419e-02 -6.7026e-02  ...  -3.9957e-03 -4.8995e-02 -2.6874e-02
 1.0078e-02  5.6950e-02  4.7811e-02  ...  -2.8043e-02 -1.5236e-02  5.6821e-02
                ...                   ⋱                   ...                
 2.8831e-02 -4.6060e-02  2.2889e-03  ...   6.2150e-02 -9.3983e-03  6.2308e-03
 7.0230e-02 -3.8726e-02  3.5802e-02  ...  -6.7231e-03  6.8262e-02  4.8362e-02
 4.1380e-03  3.4188e-02 -1.2079e-02  ...   9.5499e-03 -2.1486e-02 -5.6828e-02
[torch.FloatTensor of size 800x200]

**小练习：想想为什么这个系数的大小是 (400, 100)**

In [24]:
lstm_input = Variable(torch.randn(10, 3, 100)) # 序列 10，batch 是 3，输入维度 50

In [25]:
out, (h, c) = lstm_seq(lstm_input) # 使用默认的全 0 隐藏状态

注意这里 LSTM 输出的隐藏状态有两个，h 和 c，就是上图中的每个 cell 之间的两个箭头，这两个隐藏状态的大小都是相同的，(num_layers * direction, batch, feature)

In [26]:
h.shape # 两层，Batch 是 3，特征是 100

torch.Size([2, 3, 200])

In [27]:
c.shape

torch.Size([2, 3, 200])

In [28]:
out.shape

torch.Size([10, 3, 200])

可以不使用默认的隐藏状态，需要传入两个张量

In [29]:
h_init = Variable(torch.randn(2, 3, 200))
c_init = Variable(torch.randn(2, 3, 200))

In [30]:
out, (h, c) = lstm_seq(lstm_input, (h_init, c_init))

In [31]:
h.shape

torch.Size([2, 3, 200])

In [32]:
c.shape

torch.Size([2, 3, 200])

In [33]:
out.shape

torch.Size([10, 3, 200])

# GRU

In [34]:
gru_seq = nn.GRU(10, 20)
gru_input = Variable(torch.randn(3, 32, 10))

out, h = gru_seq(gru_input)

In [35]:
gru_seq.weight_hh_l0

Parameter containing:
-0.0502 -0.2164  0.1631  ...   0.1759  0.2210  0.0295
-0.1734  0.1791 -0.1977  ...   0.0644  0.0639  0.0088
 0.1789  0.0470 -0.0069  ...  -0.0223 -0.2062  0.2129
          ...             ⋱             ...          
-0.2094 -0.0395 -0.0309  ...   0.0344 -0.1244  0.0440
 0.2078 -0.1413 -0.1278  ...   0.1132  0.0830 -0.1922
-0.1499 -0.1484 -0.0728  ...   0.0370 -0.1638 -0.0094
[torch.FloatTensor of size 60x20]

In [36]:
h.shape

torch.Size([1, 32, 20])

In [37]:
out.shape

torch.Size([3, 32, 20])

## RNN for MNIST

In [38]:
import sys
sys.path.append('..')

import torch
from torch.autograd import Variable
from torch import nn
from torch.utils.data import DataLoader

from torchvision import transforms as tfs
from torchvision.datasets import MNIST

In [39]:
# 定义数据
data_tf = tfs.Compose([
    tfs.ToTensor(),
    tfs.Normalize([0.5], [0.5]) # 标准化
])

train_set = MNIST('./../../dataset', train=True, transform=data_tf)
test_set = MNIST('./../../dataset', train=False, transform=data_tf)

train_data = DataLoader(train_set, 64, True, num_workers=4)
test_data = DataLoader(test_set, 128, False, num_workers=4)

In [40]:
# 定义模型
class rnn_classify(nn.Module):
    def __init__(self, in_feature=28, hidden_feature=100, num_class=10, num_layers=2):
        super(rnn_classify, self).__init__()
        self.rnn = nn.LSTM(in_feature, hidden_feature, num_layers) # 使用两层 lstm
        self.classifier = nn.Linear(hidden_feature, num_class) # 将最后一个 rnn 的输出使用全连接得到最后的分类结果
        
    def forward(self, x):
        '''
        x 大小为 (batch, 1, 28, 28)，所以我们需要将其转换成 RNN 的输入形式，即 (28, batch, 28)
        '''
        x = x.squeeze() # 去掉 (batch, 1, 28, 28) 中的 1，变成 (batch, 28, 28)
        x = x.permute(2, 0, 1) # 将最后一维放到第一维，变成 (28, batch, 28)
        out, _ = self.rnn(x) # 使用默认的隐藏状态，得到的 out 是 (28, batch, hidden_feature)
        out = out[-1, :, :] # 取序列中的最后一个，大小是 (batch, hidden_feature)
        out = self.classifier(out) # 得到分类结果
        return out

In [41]:
net = rnn_classify()
criterion = nn.CrossEntropyLoss()

optimzier = torch.optim.Adadelta(net.parameters(), 1e-1)

In [42]:
# 开始训练
from train_utils import train
train(net, train_data, test_data, 10, optimzier, criterion)

Process Process-3:
Process Process-1:
Process Process-4:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 114, in run
    self.run()
    self.run()
    self._target(*self._args, **self._kwargs)
    self.run()
  File "/System/Library/F

KeyboardInterrupt: 